In [1]:
from rethon import GlobalREEnsembleGenerator, ReflectiveEquilibrium, REState, StandardGlobalReflectiveEquilibrium
from rethon.util import standard_model_params_varied_alphas
from tau.util import create_random_argument_list
from tau.util import random_position_as_set

from tau import StandardPosition, DAGDialecticalStructure, DialecticalStructure, BitarrayPosition, BDDDialecticalStructure
from tau.util import inferential_density

from os import getcwd, path
from typing import List

import pandas as pd
from pandas import Series

from ast import literal_eval
import tarfile

### Custom Ensemble Generator Class
Excluding unused objects for efficiency

In [2]:
class CustomREEnsembleGenerator(GlobalREEnsembleGenerator):
    
    def init_tau_fields(self, tau: DialecticalStructure):

        self.add_obj('ds_infer_dens', inferential_density(tau))
        #self.add_obj('n_premises', [len(arg) for arg in tau.get_arguments()])
        #self.add_obj('principles', get_principles(tau.get_arguments()))
        #tau_truths = tau.closure(BitarrayPosition(set(),
                                                  #tau.sentence_pool().size()))
        #self.add_obj('tau_truths', tau_truths)
        #self.add_obj('tau_falsehoods', BitarrayPosition({-prop for prop in tau_truths.as_set()},
                                                        #tau.sentence_pool().size()))    def init_re_start_fields(self, reflective_equilibrium: ReflectiveEquilibrium,
 
            
    def init_re_start_fields(self, reflective_equilibrium: ReflectiveEquilibrium,
                             dialectical_structure: DialecticalStructure):

        init_coms = reflective_equilibrium.state().initial_commitments()

        #if dialectical_structure.is_consistent(init_coms):
            #init_com_min_ax_bases = _get_min_sets([axioms.as_set() for axioms in
                                                    #dialectical_structure.axioms(
                                                    #init_coms,
                                                    #init_coms.subpositions())])
        #else:
            #init_com_min_ax_bases = np.nan
        #self.add_obj('init_com_min_ax_bases', init_com_min_ax_bases)
        
        # Global optima
        global_optima = list(reflective_equilibrium.global_optima(init_coms))
        self.add_obj('global_optima', global_optima)

        # RE-states
        re_states = {(theory, commitments) for (theory, commitments) in global_optima
                     if dialectical_structure.is_consistent(BitarrayPosition.union({commitments, theory}))}
        self.add_obj('re_states', re_states)

        # full RE-states
        full_re_states = {(theory, commitments) for (theory, commitments) in global_optima
                          if dialectical_structure.closure(theory) == commitments}
        self.add_obj('full_re_states', full_re_states)

        
    def init_ensemble_fields(self, ensemble_states: List[REState], dialectical_structure: DialecticalStructure):

        self.add_obj('n_branches', len(ensemble_states))

        # get all fixed_points of the process
        branched_theories = [branch_state.last_theory() for branch_state in ensemble_states]
        branched_commitments = [branch_state.last_commitments() for branch_state in ensemble_states]
        fixed_points = list(Series(zip(branched_theories, branched_commitments)).unique())
        self.add_obj('fixed_points', fixed_points)
        
    def init_re_final_fields(self, reflective_equilibrium: ReflectiveEquilibrium,
                             dialectical_structure: DialecticalStructure):
        pass
    
        #fp_comms_min_ax_bases = _fp_comms_min_ax_bases(dialectical_structure,
                                                                    #reflective_equilibrium.state().last_commitments())
        #self.add_obj('fixed_point_coms_min_ax_bases', fp_comms_min_ax_bases)
        #fp_comms_min_ax_bases_th = _fp_comms_min_ax_bases_given_theory(dialectical_structure,
                                                     #reflective_equilibrium.state().last_commitments(),
                                                     #reflective_equilibrium.state().last_theory())
        #self.add_obj('fixed_point_coms_min_ax_bases_theory', fp_comms_min_ax_bases_th)
        
        # remove unused fields from ensemble generators

In [3]:
item_remove_list = [#'model_name'
                    #'ds',
                    #'n_sentence_pool',
                    'ds_arg_size',
                    #'ds_infer_dens',
                    #'ds_n_consistent_complete_positions',
                    'ds_mean_prem',
                    'ds_variance_prem',
                    'tau_truths',
                    'principles',
                    #'account_penalties',
                    #'faithfulness_penalties',
                    #'weight_account',
                    #'weight_systematicity',
                    #'weight_faithfulness',
                    #'init_coms',
                    'init_coms_size',
                    'init_coms_n_tau_truths',
                    'init_coms_n_tau_falsehoods',
                    'init_coms_n_consistent_complete_positions',
                    #'init_coms_dia_consistent',
                    'init_coms_closed',
                    #'fixed_point_coms',
                    'fixed_point_coms_size',
                    'fixed_point_coms_n_tau_truths',
                    'fixed_point_coms_n_tau_falsehoods',
                    'fixed_point_coms_closed',
                    #'fixed_point_coms_consistent',
                    'fixed_point_coms_n_consistent_complete_positions',
                    #'fixed_point_theory',
                    'fixed_point_theory_closure',
                    'init_coms_min_ax_bases',
                    'n_init_coms_min_ax_base',
                    'achievements_evolution',
                    #'fixed_point_dia_consistent',
                    'init_final_coms_simple_hamming',
                    'init_final_coms_hamming',
                    'init_final_coms_contradictions',
                    'init_final_coms_expansions',
                    'init_final_coms_contractions',
                    'init_final_coms_identities',
                    'random_choices',
                    'n_random_choices',
                    'coms_evolution',
                    'theory_evolution',
                    'process_length',
                    #'n_branches',
                    #'fixed_points',
                    #'n_fixed_points',
                    #'fp_coms_consistent',
                    #'fp_union_consistent',
                    'fp_account',
                    'fp_faithfulness',
                    #'fixed_point_is_global_optimum',
                    #'fixed_point_is_re_state',
                    #'fixed_point_is_full_re_state',
                    #'global_optima',
                    #'n_global_optima',
                    #'go_coms_consistent',
                    #'go_union_consistent',
                    #'go_full_re_state',
                    'go_account',
                    'go_faithfulness',
                    're_states',
                    'n_re_states',
                    #'full_re_states',
                    #'n_full_re_states',
                    'fixed_point_coms_min_ax_bases',
                    'n_fixed_point_coms_min_ax_base',
                    'fixed_point_coms_min_ax_bases_theory',
                    'n_fixed_point_coms_min_ax_base_theory',
                    'fixed_point_theory_axioms',
                    #'go_fixed_point',
                    #'fp_full_re_state',
                    #'fp_global_optimum'
                    ]

### Ensemble Generation: Convergence - Full Spectrum

In [13]:
# sentence pool
n = 7

# standard example
arguments = [[1, 3],[1, 4],[1, 5],[1, -6], [2, -4],[2, 5],[2, 6],[2, 7]]


arguments_list = [arguments]

dia = DAGDialecticalStructure(n, arguments)


print(arguments)
print(inferential_density(dia))

[[1, 3], [1, 4], [1, 5], [1, -6], [2, -4], [2, 5], [2, 6], [2, 7]]
0.26143928550824114


In [14]:
# arguments for each dialectical structure
arguments_list = [arguments]

In [15]:
# full spectrum of commitments
coms_list = [pos.as_set() for pos in dia.minimally_consistent_positions() if pos.size()]
print(len(coms_list))

2186


In [24]:
# model parameters
configurations = [
                  [0.35, 0.55, 0.10],
                  #[0.55, 0.35, 0.10],
                  #[0.70, 0.20, 0.10],
                  #[0.55, 0.20, 0.25],
                  #[0.46, 0.10, 0.44],
                  #[0.10, 0.55, 0.35],
                  #[0.10, 0.35, 0.55],
                  ]




model_parameters_list = []

for configuration in configurations:
    model_parameters = StandardGlobalReflectiveEquilibrium.default_model_parameters()
    
    model_parameters["weights"]["account"] = configuration[0]
    model_parameters["weights"]["systematicity"] = configuration[1]
    model_parameters["weights"]["faithfulness"] = configuration[2]
    
    model_parameters_list.append(model_parameters)


In [43]:
model_parameters_list

[{'weights': {'account': 0.35, 'systematicity': 0.55, 'faithfulness': 0.1},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.55, 'systematicity': 0.35, 'faithfulness': 0.1},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.7, 'systematicity': 0.2, 'faithfulness': 0.1},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.55, 'systematicity': 0.2, 'faithfulness': 0.25},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.46, 'systematicity': 0.1, 'faithfulness': 0.44},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.1, 'systematicity': 0.55, 'faithfulness': 0.35},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penal

In [39]:
# standard example

# default implementation:
implementations = [{'tau_module_name': 'tau',
                    'position_class_name':'StandardPosition',
                    'dialectical_structure_class_name': 'DAGDialecticalStructure',
                    'rethon_module_name': 'rethon',
                    'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'}]


    

ensemble_gen = CustomREEnsembleGenerator(arguments_list = arguments_list, 
                                     n_sentence_pool = n,
                                     initial_commitments_list = coms_list,
                                     model_parameters_list = model_parameters_list,
                                     implementations = implementations,
                                     create_branches = True)
# remove unused fields
for item in item_remove_list:
    ensemble_gen.remove_item(item)



ensemble_gen.ensemble_items_to_csv(
                              output_file_name = 'data_full_spectrum_convergence_std_ex.csv',
                              output_dir_name = path.join(getcwd(), "data"),
                              archive = True, # save the csv as archived tar.gz
                              save_preliminary_results = True, # will create preliminary csv-data sets 
                              preliminary_results_interval = 100)

In [25]:
# random dialectical structures

n=7

for i in range(21,31):
    
    # one dialectical structure in inferential density range
    while True:

        arguments = create_random_argument_list(n_arguments_min=n-2, n_arguments_max=n+1,
                                    n_sentences=n, n_premises_max=2)
        
        dia = BDDDialecticalStructure(n, arguments)
        
        # check inferential density and usage of all sentences
        if (0.15<=inferential_density(dia)<=0.5) and (len(set(abs(s) for arg in arguments for s in arg))==n):
            
            # ensure that there are no tau-truths
            if dia.closure(StandardPosition.from_set(set(), 3)).size()==0:
            
                arguments_list = [arguments]
                print(arguments)
                break
            
    # default implementation:
    implementations = [{'tau_module_name': 'tau',
                        'position_class_name':'StandardPosition',
                        'dialectical_structure_class_name': 'DAGDialecticalStructure',
                        'rethon_module_name': 'rethon',
                        'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'}]




    ensemble_gen = CustomREEnsembleGenerator(arguments_list = arguments_list, 
                                         n_sentence_pool = n,
                                         initial_commitments_list = coms_list,
                                         model_parameters_list = model_parameters_list,
                                         implementations = implementations,
                                         create_branches = True)
    # remove unused fields
    for item in item_remove_list:
        ensemble_gen.remove_item(item)



    ensemble_gen.ensemble_items_to_csv(
                                  output_file_name = 'data_full_spectrum_convergence_rand_ex_'+str(i)+ '.csv',
                                  output_dir_name = path.join(getcwd(), "data"),
                                  archive = True, # save the csv as archived tar.gz
                                  save_preliminary_results = True, # will create preliminary csv-data sets 
                                  preliminary_results_interval = 100)

[[-6, -1, -4], [5, 7, 6], [-3, 2, 6], [-7, 1], [1, -5, -6], [-6, -4, -5], [5, -3]]
[[2, -3], [4, 3], [-6, 7, 2], [-5, -3], [2, -6, 3], [-1, 4, -5], [-4, 7]]
[[1, -7], [-6, -2, -1], [3, 4, -6], [6, 7, -2], [-2, 5, 1], [-1, 7]]
[[-3, -6], [1, -7, -3], [4, -2, 6], [3, -2, 7], [-7, 5, 3], [-2, 6], [1, -4]]
[[3, 2], [5, -2], [1, -2], [-4, -5], [7, -6, -4]]
[[6, -7, 5], [-3, -5], [1, 5], [2, -7], [4, 3], [4, -2, -5], [7, -6, 1], [2, -6]]
[[-5, 2, -7], [4, 6, -7], [-1, -3, 6], [4, 5, -6], [2, 6]]
[[-6, 1], [-5, 7, -6], [-3, -7], [-2, -5], [7, 4, 1], [-2, 1]]
[[-7, -5], [3, -1, 5], [2, 7], [-6, -7], [4, -3], [-7, 3]]
[[-3, -1, -2], [7, 5, -3], [-6, -1], [-4, -7], [2, -3]]


### Adding Additional Weight Configurations to Existing Full Spectrum Data

In [4]:
data_dir = path.join(getcwd(), "data")

In [6]:
# loading the data

data_file_name = 'data_full_spectrum_convergence_rand_ex_all_configs.csv.tar.gz'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    re_data = pd.read_csv(path.join(data_dir, file_name))

else:
    re_data = pd.read_csv(path.join(data_dir,data_file_name))

print(re_data.columns)
re_data.shape

Index(['model_name', 'ds', 'n_sentence_pool', 'ds_infer_dens',
       'ds_n_consistent_complete_positions', 'account_penalties',
       'faithfulness_penalties', 'weight_account', 'weight_systematicity',
       'weight_faithfulness', 'init_coms', 'init_coms_dia_consistent',
       'fixed_point_coms', 'fixed_point_coms_consistent', 'fixed_point_theory',
       'fixed_point_dia_consistent', 'n_branches', 'fixed_points',
       'n_fixed_points', 'fp_coms_consistent', 'fp_union_consistent',
       'fixed_point_is_global_optimum', 'fixed_point_is_re_state',
       'fixed_point_is_full_re_state', 'global_optima', 'n_global_optima',
       'go_coms_consistent', 'go_union_consistent', 'go_full_re_state',
       'full_re_states', 'n_full_re_states', 'go_fixed_point',
       'fp_full_re_state', 'fp_global_optimum', 'configuration'],
      dtype='object')


(327900, 35)

In [16]:
# model parameters for loop below
configurations = [#[0.35, 0.55, 0.10],
                  #[0.55, 0.35, 0.10],
                  #[0.70, 0.20, 0.10],
                  #[0.55, 0.20, 0.25],
                  #[0.46, 0.10, 0.44],
                  [0.10, 0.55, 0.35],
                  [0.10, 0.35, 0.55],
                  ]

model_parameters_list = []

for configuration in configurations:
    model_parameters = StandardGlobalReflectiveEquilibrium.default_model_parameters()
    
    model_parameters["weights"]["account"] = configuration[0]
    model_parameters["weights"]["systematicity"] = configuration[1]
    model_parameters["weights"]["faithfulness"] = configuration[2]
    
    model_parameters_list.append(model_parameters)

In [17]:
# full spectrum of commitments
coms_list = [pos.as_set() for pos in dia.minimally_consistent_positions() if pos.size()]
print(len(coms_list))

2186


In [18]:
# all dialectical structures
arguments_list = [literal_eval(ds) for ds in re_data["ds"].unique()]
print(len(arguments_list))

30


In [19]:
n=7

In [20]:
# default implementation:
implementations = [{'tau_module_name': 'tau',
                    'position_class_name':'StandardPosition',
                    'dialectical_structure_class_name': 'DAGDialecticalStructure',
                    'rethon_module_name': 'rethon',
                    'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'}]

ensemble_gen = CustomREEnsembleGenerator(arguments_list = arguments_list, 
                                     n_sentence_pool = n,
                                     initial_commitments_list = coms_list,
                                     model_parameters_list = model_parameters_list,
                                     implementations = implementations,
                                     create_branches = True)

# remove unused fields
for item in item_remove_list:
    ensemble_gen.remove_item(item)
    
ensemble_gen.ensemble_items_to_csv(
                              output_file_name = 'data_full_spectrum_convergence_rand_ex_add_config2.csv',
                              output_dir_name = path.join(getcwd(), "data"),
                              archive = True, # save the csv as archived tar.gz
                              save_preliminary_results = True, # will create preliminary csv-data sets 
                              preliminary_results_interval = 100)

In [21]:
# loading the data

data_file_name = 'data_full_spectrum_convergence_rand_ex_all_configs.csv.tar.gz'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    re_data = pd.read_csv(path.join(data_dir, file_name))

else:
    re_data = pd.read_csv(path.join(data_dir,data_file_name))

print(re_data.columns)
re_data.shape


Index(['model_name', 'ds', 'n_sentence_pool', 'ds_infer_dens',
       'ds_n_consistent_complete_positions', 'account_penalties',
       'faithfulness_penalties', 'weight_account', 'weight_systematicity',
       'weight_faithfulness', 'init_coms', 'init_coms_dia_consistent',
       'fixed_point_coms', 'fixed_point_coms_consistent', 'fixed_point_theory',
       'fixed_point_dia_consistent', 'n_branches', 'fixed_points',
       'n_fixed_points', 'fp_coms_consistent', 'fp_union_consistent',
       'fixed_point_is_global_optimum', 'fixed_point_is_re_state',
       'fixed_point_is_full_re_state', 'global_optima', 'n_global_optima',
       'go_coms_consistent', 'go_union_consistent', 'go_full_re_state',
       'full_re_states', 'n_full_re_states', 'go_fixed_point',
       'fp_full_re_state', 'fp_global_optimum', 'configuration'],
      dtype='object')


(327900, 35)

In [22]:
# loading the data

data_file_name = 'data_full_spectrum_convergence_rand_ex_add_config2.csv.tar.gz'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    re_data2 = pd.read_csv(path.join(data_dir, file_name))

else:
    re_data2 = pd.read_csv(path.join(data_dir,data_file_name))

print(re_data2.columns)
re_data2.shape

Index(['model_name', 'ds', 'n_sentence_pool', 'ds_infer_dens',
       'ds_n_consistent_complete_positions', 'account_penalties',
       'faithfulness_penalties', 'weight_account', 'weight_systematicity',
       'weight_faithfulness', 'init_coms', 'init_coms_dia_consistent',
       'fixed_point_coms', 'fixed_point_coms_consistent', 'fixed_point_theory',
       'fixed_point_dia_consistent', 'n_branches', 'fixed_points',
       'n_fixed_points', 'fp_coms_consistent', 'fp_union_consistent',
       'fixed_point_is_global_optimum', 'fixed_point_is_re_state',
       'fixed_point_is_full_re_state', 'global_optima', 'n_global_optima',
       'go_coms_consistent', 'go_union_consistent', 'go_full_re_state',
       'full_re_states', 'n_full_re_states', 'go_fixed_point',
       'fp_full_re_state', 'fp_global_optimum'],
      dtype='object')


(131160, 34)

In [23]:
re_data3 = pd.concat([re_data, re_data2])

In [24]:
re_data3.shape

(459060, 35)

In [27]:
re_data3.groupby(["weight_account", "weight_systematicity"])["ds"].size().reset_index()

,weight_account,weight_systematicity,ds
0,0.10,0.35,65580
1,0.10,0.55,65580
2,0.35,0.55,65580
3,0.46,0.10,65580
4,0.55,0.20,65580
5,0.55,0.35,65580
6,0.70,0.20,65580


In [28]:
data_dir = path.join(getcwd(), "data")
file_name="data_full_spectrum_convergence_rand_ex_all_configs2.csv"

# write results to a .csv.file
re_data3.to_csv(path.join(data_dir, file_name), index=False)

# add to a tar file
tar_file = path.join(data_dir, file_name + '.tar.gz')
with tarfile.open(tar_file, "w:gz") as tar:
    tar.add(path.join(data_dir, file_name), recursive=False, arcname=file_name)

## Ensemble Generation: Two Point Convergence

In [5]:
# model parameters for loop below
configurations = [[0.35, 0.55, 0.10],
                  #[0.55, 0.35, 0.10],
                  #[0.70, 0.20, 0.10],
                  #[0.55, 0.20, 0.25],
                  #[0.46, 0.10, 0.44],
                  #[0.10, 0.55, 0.35],
                  #[0.10, 0.35, 0.55],
                  ]

model_parameters_list = []

for configuration in configurations:
    model_parameters = StandardGlobalReflectiveEquilibrium.default_model_parameters()
    
    model_parameters["weights"]["account"] = configuration[0]
    model_parameters["weights"]["systematicity"] = configuration[1]
    model_parameters["weights"]["faithfulness"] = configuration[2]
    
    model_parameters_list.append(model_parameters)

In [7]:
n_dia_structures = 400


for n_sentence_pool in [9]:
    
    print(n_sentence_pool)

    # loop
    for j in range(n_dia_structures):

        arguments_list = []

        # one dialectical structure in inferential density range
        while True:

            arguments = create_random_argument_list(n_arguments_min=n_sentence_pool-2,
                                                    n_arguments_max=n_sentence_pool+1,
                                                    n_sentences=n_sentence_pool, 
                                                    n_premises_max=2)

            dia = DAGDialecticalStructure(n_sentence_pool, arguments)

            # check inferential density and usage of all sentences
            if (0.15<=inferential_density(dia)<=0.5) and (len(set(abs(s) for arg in arguments for s in arg))==n_sentence_pool):
                arguments_list = [arguments]
                #print(arguments)
                break

        coms_list = []

        while not coms_list:

            init_coms1 = StandardPosition.from_set(random_position_as_set(n_sentence_pool), n_sentence_pool)
            init_coms2 = StandardPosition.from_set(random_position_as_set(n_sentence_pool), n_sentence_pool)

            # ensure non-identical initial committments
            if init_coms1 != init_coms2:

                coms_list = [init_coms1.as_set(), init_coms2.as_set()]

        # default implementation:
        implementations = [{'tau_module_name': 'tau',
                            'position_class_name':'StandardPosition',
                            'dialectical_structure_class_name': 'DAGDialecticalStructure',
                            'rethon_module_name': 'rethon',
                            'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'}]




        ensemble_gen = CustomREEnsembleGenerator(arguments_list = arguments_list, 
                                             n_sentence_pool = n_sentence_pool,
                                             initial_commitments_list = coms_list,
                                             model_parameters_list = model_parameters_list,
                                             implementations = implementations,
                                             create_branches = True)
        # remove unused fields
        for item in item_remove_list:
            ensemble_gen.remove_item(item)



        ensemble_gen.ensemble_items_to_csv(
                                      output_file_name = 'data_two_point_convergence_sp_'+str(n_sentence_pool)+'_05.csv',
                                      output_dir_name = path.join(getcwd(), "data"),
                                      archive = False, # save the csv as archived tar.gz
                                      save_preliminary_results = True, # will create preliminary csv-data sets 
                                      preliminary_results_interval = 100,
                                      append = True)

9


In [12]:
# concatenation and saving as tar file
import tarfile
from itertools import product

data_dir = path.join(getcwd(), "data")

dfs = []
for file_name in ["data_two_point_convergence_sp_{}_{}.csv".format(i,j) 
                  for (i,j) in product(range(6,10), ["0{}".format(i) for i in range(6)])]:

    dfs.append(pd.read_csv(path.join(data_dir, file_name)))

re_data = pd.concat(dfs, ignore_index=True)

file_name = "data_two_point_convergence_sp_6789_00.csv"

re_data.to_csv(path.join(data_dir, file_name), index=False)

tar_file = path.join(data_dir, file_name + '.tar.gz')
with tarfile.open(tar_file, "w:gz") as tar:
    tar.add(path.join(data_dir, file_name), recursive=False, arcname=file_name)

In [11]:
# sentence pool
n = 7

# standard example
arguments = [[1, 3],[1, 4],[1, 5],[1, -6], [2, -4],[2, 5],[2, 6],[2, 7]]


arguments_list = [arguments]

dia = BDDDialecticalStructure(n, arguments)
re= StandardGlobalReflectiveEquilibrium(dia)

In [12]:
from math import floor




In [13]:
# model parameters for loop below
configurations = [[0.35, 0.55, 0.10],
                  #[0.55, 0.35, 0.10],
                  #[0.70, 0.20, 0.10],
                  #[0.55, 0.20, 0.25],
                  #[0.46, 0.10, 0.44],
                  #[0.10, 0.55, 0.35],
                  #[0.10, 0.35, 0.55],
                  ]

model_parameters_list = []

for configuration in configurations:
    model_parameters = StandardGlobalReflectiveEquilibrium.default_model_parameters()
    
    model_parameters["weights"]["account"] = configuration[0]
    model_parameters["weights"]["systematicity"] = configuration[1]
    model_parameters["weights"]["faithfulness"] = configuration[2]
    
    model_parameters_list.append(model_parameters)

In [14]:
n_sentence_pool=7

min_init_coms_size = floor(n_sentence_pool/2)

n_pairs = 50

pair_list = []

for target_hamming_dist in range(1,2*n_sentence_pool):
    
    min_init_coms_size = max(round(target_hamming_dist/2, 0), min_init_coms_size)

    for _ in range(n_pairs):
        
        coms_list = []
        
        while not coms_list:

                    init_coms1 = StandardPosition.from_set(random_position_as_set(n_sentence_pool), n_sentence_pool)

                    if init_coms1.size()>=min_init_coms_size:
                        #print(init_coms1.size())

                        init_coms2 = StandardPosition.from_set(random_position_as_set(n_sentence_pool), n_sentence_pool)

                        if init_coms2.size()>=min_init_coms_size:
                            #print(init_coms2.size())

                            # ensure non-identical initial committments
                            if re.hamming_distance (init_coms1, init_coms2, [0,1,1,2])==target_hamming_dist:

                                coms_list = [init_coms1.as_set(), init_coms2.as_set()]
                                
                                pair_list.append(coms_list)
                                #print(coms_list, target_hamming_dist)


print(len(pair_list))

for i, coms_list in enumerate(pair_list):
    
        print(i)
        # one dialectical structure in inferential density range
        while True:

            arguments = create_random_argument_list(n_arguments_min=n_sentence_pool-2,
                                                    n_arguments_max=n_sentence_pool+1,
                                                    n_sentences=n_sentence_pool, 
                                                    n_premises_max=2)

            dia = DAGDialecticalStructure(n_sentence_pool, arguments)

            # check inferential density and usage of all sentences
            if (0.15<=inferential_density(dia)<=0.5) and (len(set(abs(s) for arg in arguments for s in arg))==n_sentence_pool):
                arguments_list = [arguments]
                #print(arguments)
                break

        

        # default implementation:
        implementations = [{'tau_module_name': 'tau',
                            'position_class_name':'StandardPosition',
                            'dialectical_structure_class_name': 'DAGDialecticalStructure',
                            'rethon_module_name': 'rethon',
                            'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'}]




        ensemble_gen = CustomREEnsembleGenerator(arguments_list = arguments_list, 
                                             n_sentence_pool = n_sentence_pool,
                                             initial_commitments_list = coms_list,
                                             model_parameters_list = model_parameters_list,
                                             implementations = implementations,
                                             create_branches = True)
        # remove unused fields
        for item in item_remove_list:
            ensemble_gen.remove_item(item)



        ensemble_gen.ensemble_items_to_csv(
                                      output_file_name = 'data_two_point_convergence_binned_sp_'+str(n_sentence_pool)+'_018.csv',
                                      output_dir_name = path.join(getcwd(), "data"),
                                      archive = False, # save the csv as archived tar.gz
                                      save_preliminary_results = True, # will create preliminary csv-data sets 
                                      preliminary_results_interval = 100,
                                      append = True)

650
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [17]:
# concatenation and saving as tar file
import tarfile
from itertools import product

data_dir = path.join(getcwd(), "data")

dfs = []
for file_name in ["data_two_point_convergence_binned_sp_7_0{}.csv".format(i) 
                  for i in range(19)]:

    dfs.append(pd.read_csv(path.join(data_dir, file_name)))

re_data = pd.concat(dfs, ignore_index=True)

file_name = "data_two_point_convergence_binned_sp_7.csv"

re_data.to_csv(path.join(data_dir, file_name), index=False)

tar_file = path.join(data_dir, file_name + '.tar.gz')
with tarfile.open(tar_file, "w:gz") as tar:
    tar.add(path.join(data_dir, file_name), recursive=False, arcname=file_name)

In [18]:
len(re_data)

26000

### Additional Configuration of Weights For Existing "Two-Point" Data

In [4]:
data_dir = path.join(getcwd(), "data")

In [6]:
# loading the data

data_file_name = 'data_two_point_convergence_binned_sp_7_all_configs.csv.tar.gz'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    re_data = pd.read_csv(path.join(data_dir, file_name))

else:
    re_data = pd.read_csv(path.join(data_dir,data_file_name))

print(re_data.columns)
re_data.shape

Index(['model_name', 'ds', 'n_sentence_pool', 'ds_infer_dens',
       'ds_n_consistent_complete_positions', 'account_penalties',
       'faithfulness_penalties', 'weight_account', 'weight_systematicity',
       'weight_faithfulness', 'init_coms', 'init_coms_dia_consistent',
       'fixed_point_coms', 'fixed_point_coms_consistent', 'fixed_point_theory',
       'fixed_point_dia_consistent', 'n_branches', 'fixed_points',
       'n_fixed_points', 'fp_coms_consistent', 'fp_union_consistent',
       'fixed_point_is_global_optimum', 'fixed_point_is_re_state',
       'fixed_point_is_full_re_state', 'global_optima', 'n_global_optima',
       'go_coms_consistent', 'go_union_consistent', 'go_full_re_state',
       'full_re_states', 'n_full_re_states', 'go_fixed_point',
       'fp_full_re_state', 'fp_global_optimum', 'configuration'],
      dtype='object')


C:\Users\afrei\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(130000, 35)

In [7]:
re_data.groupby("ds")["init_coms"].nunique().describe()

count    13000.0
mean         2.0
std          0.0
min          2.0
25%          2.0
50%          2.0
75%          2.0
max          2.0
Name: init_coms, dtype: float64

In [8]:
# model parameters for loop below
configurations = [#[0.35, 0.55, 0.10],
                  #[0.55, 0.35, 0.10],
                  #[0.70, 0.20, 0.10],
                  #[0.55, 0.20, 0.25],
                  #[0.46, 0.10, 0.44],
                  [0.10, 0.55, 0.35],
                  [0.10, 0.35, 0.55],
                  ]

model_parameters_list = []

for configuration in configurations:
    model_parameters = StandardGlobalReflectiveEquilibrium.default_model_parameters()
    
    model_parameters["weights"]["account"] = configuration[0]
    model_parameters["weights"]["systematicity"] = configuration[1]
    model_parameters["weights"]["faithfulness"] = configuration[2]
    
    model_parameters_list.append(model_parameters)

In [9]:
from IPython.display import clear_output

In [10]:
n_sentence_pool = 7

In [11]:
for i, ds in enumerate(re_data["ds"].unique()):
    
    clear_output(wait=True)
    print(i)

    
    arguments_list = []
    arguments_list.append(literal_eval(ds))
    
    coms_list = []
    
    for coms in re_data[re_data["ds"]==ds]["init_coms"].unique():
        coms_list.append(literal_eval(coms))

    # default implementation:
    implementations = [{'tau_module_name': 'tau',
                        'position_class_name':'StandardPosition',
                        'dialectical_structure_class_name': 'DAGDialecticalStructure',
                        'rethon_module_name': 'rethon',
                        'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'}]
    
    ensemble_gen = CustomREEnsembleGenerator(arguments_list = arguments_list, 
                                             n_sentence_pool = n_sentence_pool,
                                             initial_commitments_list = coms_list,
                                             model_parameters_list = model_parameters_list,
                                             implementations = implementations,
                                             create_branches = True)
    # remove unused fields
    for item in item_remove_list:
        ensemble_gen.remove_item(item)
        
    ensemble_gen.ensemble_items_to_csv(
                                      output_file_name = 'data_two_point_convergence_binned_sp_7_add_config.csv',
                                      output_dir_name = path.join(getcwd(), "data"),
                                      archive = False, # save the csv as archived tar.gz
                                      save_preliminary_results = True, # will create preliminary csv-data sets 
                                      preliminary_results_interval = 500,
                                      append = True)

    

12999


In [13]:
# loading the data

data_file_name = 'data_two_point_convergence_binned_sp_7_all_configs.csv.tar.gz'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    re_data = pd.read_csv(path.join(data_dir, file_name))

else:
    re_data = pd.read_csv(path.join(data_dir,data_file_name))

print(re_data.columns)
re_data.shape

Index(['model_name', 'ds', 'n_sentence_pool', 'ds_infer_dens',
       'ds_n_consistent_complete_positions', 'account_penalties',
       'faithfulness_penalties', 'weight_account', 'weight_systematicity',
       'weight_faithfulness', 'init_coms', 'init_coms_dia_consistent',
       'fixed_point_coms', 'fixed_point_coms_consistent', 'fixed_point_theory',
       'fixed_point_dia_consistent', 'n_branches', 'fixed_points',
       'n_fixed_points', 'fp_coms_consistent', 'fp_union_consistent',
       'fixed_point_is_global_optimum', 'fixed_point_is_re_state',
       'fixed_point_is_full_re_state', 'global_optima', 'n_global_optima',
       'go_coms_consistent', 'go_union_consistent', 'go_full_re_state',
       'full_re_states', 'n_full_re_states', 'go_fixed_point',
       'fp_full_re_state', 'fp_global_optimum', 'configuration'],
      dtype='object')


C:\Users\afrei\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(130000, 35)

In [15]:
# loading the data

data_file_name = 'data_two_point_convergence_binned_sp_7_add_config.csv'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    re_data2 = pd.read_csv(path.join(data_dir, file_name))

else:
    re_data2 = pd.read_csv(path.join(data_dir,data_file_name))

print(re_data2.columns)
re_data2.shape

Index(['model_name', 'ds', 'n_sentence_pool', 'ds_infer_dens',
       'ds_n_consistent_complete_positions', 'account_penalties',
       'faithfulness_penalties', 'weight_account', 'weight_systematicity',
       'weight_faithfulness', 'init_coms', 'init_coms_dia_consistent',
       'fixed_point_coms', 'fixed_point_coms_consistent', 'fixed_point_theory',
       'fixed_point_dia_consistent', 'n_branches', 'fixed_points',
       'n_fixed_points', 'fp_coms_consistent', 'fp_union_consistent',
       'fixed_point_is_global_optimum', 'fixed_point_is_re_state',
       'fixed_point_is_full_re_state', 'global_optima', 'n_global_optima',
       'go_coms_consistent', 'go_union_consistent', 'go_full_re_state',
       'full_re_states', 'n_full_re_states', 'go_fixed_point',
       'fp_full_re_state', 'fp_global_optimum'],
      dtype='object')


(52000, 34)

In [16]:


re_data3 = pd.concat([re_data, re_data2], ignore_index=True)

file_name = "data_two_point_convergence_binned_sp_7_all_configs2.csv"

re_data3.to_csv(path.join(data_dir, file_name), index=False)

tar_file = path.join(data_dir, file_name + '.tar.gz')
with tarfile.open(tar_file, "w:gz") as tar:
    tar.add(path.join(data_dir, file_name), recursive=False, arcname=file_name)